# Intro to Agent Development Kit [ADK]

## Overview
This notebook provides a foundational introduction to creating a simple agent using the Google Agent Framework (ADK - Agent Development Kit). In this example, we will create an agent named "agent_basic" whose primary function is to inform users that it was built using the Google Agent Framework when asked about its creation. This showcases how to embed specific knowledge and responses directly into the agent's definition. By exploring this basic agent creation, you will gain an understanding of the fundamental building blocks and configuration options available within the Google Agent Framework for developing more sophisticated and specialized agents.

## Get started

In [ ]:
!pip install google-adk

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

In [ ]:
import os
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = "1"
os.environ['GOOGLE_CLOUD_PROJECT'] ="" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
os.environ['GOOGLE_CLOUD_LOCATION'] ="us-central1"

## Import libraries

Import tutorial libraries.

In [ ]:
# Import libraries from the Agent Framework
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

In [ ]:
# --- Constants ---
APP_NAME = "base_agent_app"
USER_ID = "dev_user_01"
SESSION_ID = "workshop_session_01"
MODEL = "gemini-2.0-flash"

In [ ]:
# Create a basic agent with instructions amd greeting only
basic_agent = Agent(
        model=MODEL,
        name="agent_basic",
        description="This agent responds to inquiries about its creation by stating it was built using the Google Agent Framework.",
        instruction="If they ask you how you were created, tell them you were created with the Google Agent Framework.",
        generate_content_config=types.GenerateContentConfig(temperature=0.2),
    )

In [ ]:
session_service = InMemorySessionService()
# Initializes an in-memory session service. This service is used to manage and persist session data for the agents,
# allowing them to maintain state across multiple interactions within the current runtime. Data stored here will
# not persist beyond the lifespan of this Python process. For production environments, a persistent session
# service (e.g., using a database) would be necessary.


session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
# Creates a new session using the session service, associating it with the application (APP_NAME),
# the current user (USER_ID), and a unique session identifier (SESSION_ID). This initializes
# the user's interaction context and stores relevant session-specific data.

runner = Runner(agent=basic_agent, app_name=APP_NAME, session_service=session_service)
# Initializes a Runner instance, which is responsible for executing the 'basic_agent' within the specified application ('APP_NAME')
# and managing its sessions using the provided 'session_service'.

### Create a utility function

In [ ]:
# Agent Interaction
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)


In [ ]:
call_agent("Who are you ?")